In [12]:
#STEP 1: Needed functions and variables
import csv
from datetime import datetime
from datetime import timedelta
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import requests
import time
from turtle import title
from turtle import width
from urllib.request import urlopen

ETH = '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'
WBTC = '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f'
LINK = '0xf97f4df75117a78c1A5a0DBb814Af92458539FB4'
UNI = '0xFa7F8980b0f1E64A2062791cc3b0871572f1F7f0'
# today_date = datetime.now().strftime("%m/%d/%Y")

#change to include other coins
coins = {ETH:'ETH/USD',WBTC:'WBTC/USD'}
WBTC_data = []
ETH_data = []

prices_url = "https://api.gmx.io/prices" #prices_json -> dict : {'addy':'price * 10**30}

def price_format(x):
    return int(x)/10**30

def included_coins(x):
    return True if x in coins else False

def get_prices():
    response = urlopen(prices_url) # store the response of URL 
    prices_json = json.loads(response.read()) # storing the JSON response
    return prices_json

def time_price_data():
    
    p_json = get_prices()
    now = datetime.now()
    current_time = now.strftime("%I:%M:%S %p")
    # print("Time:", current_time)
    
    for addy in p_json:
        if addy in coins:
            # print(coins[addy],price_format(p_json[addy]))
            if coins[addy] == 'WBTC/USD':
                WBTC_data.append([datetime.now().strftime("%m/%d/%Y"),current_time,'WBTC/USD',price_format(p_json[addy])])
            if coins[addy] == 'ETH/USD':
                ETH_data.append([datetime.now().strftime("%m/%d/%Y"),current_time,'ETH/USD',price_format(p_json[addy])])
    return

In [37]:
#STEP 2: Acquiring LIVE data from GMX's API
green_light = True
print('Record Start: ' + datetime.now().strftime("%I:%M:%S %p"))
while green_light:
    try:
        time_price_data()
        time.sleep(0.5)
    except KeyboardInterrupt:
        print('Record End: ' + datetime.now().strftime("%I:%M:%S %p"))
        break

Record Start: 10:16:18 PM
Record End: 10:16:33 PM


In [38]:
#STEP 3: Creating CSV, Dataframe, and plots for analysis and visuals

def create_df_file(coin):
    global df
    col_names = [['Date','Time','Pair','Price']]

    #Formatting data and creating an CSV file and dataframe
    txt_file_str = str(coin) +'_data.txt'
    with open(txt_file_str,'w') as f:
        write = csv.writer(f)
        write.writerows(col_names)
        if coin == 'WBTC':
            write.writerows(WBTC_data)
        elif coin == 'ETH':
            write.writerows(ETH_data)

    read_file = pd.read_csv(txt_file_str)
    csv_file_str = str(coin) +'_data.csv'
    read_file.to_csv(csv_file_str, index=None)
    df = pd.DataFrame(pd.read_csv(txt_file_str))

    f.close()
    
def plot_rate(coin,rate):  #rate(in seconds) is the time-scaling interval of VIEWING the data
    
    create_df_file(coin)
    
    opens = {}
    closes = {}
    highs = {}
    lows = {}
    average = 0
    range_values = []

    dt = rate  #time interval(1 for 1s, 60 for 1m, 300 for 5m)

    # prices = df.loc[:,'Price']
    for i in range(df.shape[0]):
        t = df.iloc[i,1] #time
        if i+dt <= df.shape[0]: #required ammo for new candle
            dp = df.iloc[i,-1] #new price

            # if i!=0:
            range_values.append(dp)
            #Open and Close values
            if i % dt == 0: #new candle
                opening = dp
                opens[t] = opening
                if i != 0: #close is last value of interval
                    close = df.iloc[i+dt-1,-1]
                    closes[t] = close
                    lows[t] = np.min(range_values) #---low at end of interval t, NOT WHEN low occurs, important distinction 
                    highs[t] = np.max(range_values)
                    # print('Min/Max:',np.min(range_values),np.max(range_values))
                    # print(i,t,range_values)
                    # print('len:',len(range_values))
                    range_values = [] #reset for next interval
        else:  
            #---Since open is first, there'll be 1 more value in open than close/low/high, so we'll pad close with current open(setting equal essentially)
            gen_time = str((datetime.strptime(list(opens.keys())[-1],'%I:%M:%S %p') + timedelta(seconds=dt)).time())
            closes[gen_time] = opens[list(opens.keys())[-1]]
            lows[gen_time] = lows[list(lows.keys())[-1]]
            highs[gen_time] = highs[list(highs.keys())[-1]]
            break

    open_data = list(opens.values())
    high_data = list(highs.values())
    low_data = list(lows.values())
    close_data = list(closes.values())
    times = list(opens.keys())


    fig = go.Figure(data=[go.Candlestick(x=times,
                                        open=open_data, 
                                        high=high_data,
                                        low=low_data, 
                                        close=close_data,
                                        name=coin)])

    fig.update_layout(
                    title={
                        'text' : coin + '/USD '+str(rate)+"s from " + df.iloc[0,1] + ' to ' + df.iloc[-1,1] + ' on ' + str(df.iloc[:,0].unique()[0]),
                        'y':0.92,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                    xaxis_title="Time",
                    yaxis_title= coin + '/USD',
                    width =1900,
                    height = 900)

    fig.show()

In [39]:
plot_rate('WBTC',1) #Check WBTC_plot_rate_1 for output

In [40]:
plot_rate('WBTC',60) #Check WBTC_plot_rate_60 for output

In [41]:
plot_rate('WBTC',300) #Check WBTC_plot_rate_300 for output